In [1]:
import numpy as np
from collections import OrderedDict as odict
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

This notebook computes the latencies and bandwidths of the three primitive function types

In [3]:
#(hardware name, number of nodes)
filesD = {
         'knl_mpi1':('knl',1), 'knl_mpi2':('knl',2), 'knl_mpi4':('knl',4),
         'skl_mpi1':('skl',1), 'skl_mpi2':('skl',2), 'skl_mpi4':('skl',4),
         'i5':('i5',1),
         'p100_mpi1':('p100',1), 'p100_mpi2':('p100',2), 'p100_mpi4':('p100',4),
         'v100_mpi1':('v100',1), 'v100_mpi2':('v100',2), 'v100_mpi4':('v100',4),
         'gtx1060':('gtx1060',1)
        }

files = odict(sorted(filesD.items(), key= lambda t : t[1][1]))
pd.set_option('precision',1)

#### Axpby and Dot Latencies
The latencies are determined by taking the minimum of the average runtimes 

#### Axpby and Dot Bandwidths
The bandwidths are determined by taking the average bandwidth of the 30 bandwidths corresponding to the 3 largest sizes.

#### Dx-Dy Latencies
As in Axpby 

#### Dx-Dy Bandwidths
Since the efficiency of the matrix-vector multiplications depends on the polynomial coefficient we should compute these bandwidths separately


In [18]:
names={'axpby':3,'dot':2,'dx':3, 'dy':3}
#ns=[3,4]
values = []
for f, v in files.items() :#{'knl_mpi2':('knl',2)}.items():
    runtimes=pd.read_csv('benchmark_'+f+'.csv', delimiter=' ')
    #add size and bandwidth columns
    runtimes.insert(0,'size', 8*runtimes['n']*runtimes['n']*runtimes['Nx']*runtimes['Ny']/1e6/v[1]) #inplace transformation
    for name,memops in names.items() :
        runtimes.insert(0,name+'_bw',runtimes['size']/1000*memops/runtimes[name])
    runtimes = runtimes.assign( dxdy=(runtimes['dx']+runtimes['dy'])/2)
    runtimes = runtimes.assign( dxdy_bw=2.0*runtimes['dx_bw']*runtimes['dy_bw']/(runtimes['dx_bw']+runtimes['dy_bw']))
    #compute one version with aggregated grouped sizes and one without
    avgruntimes=runtimes.groupby(['n', 'Nx','Ny','size']).agg(['mean', 'std'])
    avgruntimes=avgruntimes.reset_index(level=['n','Nx','Ny','size'])
    avgruntimes.sort_values(by='size',inplace=True) #sort by size
    runtimes.sort_values(by='size',inplace=True)
    ##first compute axpby and dot latencies and bandwidths 
    nmax = 3
    s =30

    line = []
    l=len(runtimes)
    line.append(v[0])
    line.append(v[1])
    line.append(avgruntimes[0:nmax][('axpby','mean')].mean()/1e-6)
    line.append(avgruntimes[0:nmax][('axpby','mean')].min()/1e-6)
    line.append(runtimes[l-s:l]['axpby_bw'].mean())
    line.append(runtimes[l-s:l]['axpby_bw'].std())
    line.append(avgruntimes[0:nmax][('dot','mean')].mean()/1e-6)
    line.append(avgruntimes[0:nmax][('dot','mean')].min()/1e-6)
    line.append(runtimes[l-s:l]['dot_bw'].mean())
    line.append(runtimes[l-s:l]['dot_bw'].std())
    ##now compute latency and bandwidths of dx and y
 
    for n in [2,3,4,5]:
        #take n
        dxdy=runtimes[runtimes['n']==n]
        #print(dxdy['n'])
        avgdxdy = avgruntimes[avgruntimes['n']==n]
        dxdy=dxdy.sort_values(by='size')
        avgdxdy=avgdxdy.sort_values(by='size') #sort by size
        #avgdxdy[avgdxdy['size']>9]
        mb=9
        Nx = 513 # compute among the four greates sizes
     
        line.append(avgdxdy[0:nmax][('dxdy','mean')].mean()/1e-6)
        line.append(avgdxdy[('dxdy','mean')].min()/1e-6)
        line.append(dxdy[dxdy['Nx']>=Nx]['dxdy_bw'].mean())
        line.append(dxdy[dxdy['Nx']>=Nx]['dxdy_bw'].std())
    #print(line)    
    values.append(line)

In [19]:
#now construct new table with values from previous cell      
tuples=[('arch','',''),('nodes','','')]        
for q in ['axpby','dot','dxdy2','dxdy3','dxdy4','dxdy5']:
    tuples.append((q,'lat','avg'))
    tuples.append((q,'lat','min'))
    tuples.append((q,'bw','avg'))
    tuples.append((q,'bw','std'))
cols=pd.MultiIndex.from_tuples(tuples)
arr = pd.DataFrame(values,index=files.keys(), columns=cols)
arr.sort_values(by='arch',inplace=True)
arr.set_index(['arch','nodes'],inplace=True)
#arr.loc[:,[('dot','bw','avg'),('dot','lat','avg')]]
arr

axpby                          dot                         \
                lat           bw             lat            bw            
                avg   min    avg      std    avg    min    avg      std   
arch    nodes                                                             
gtx1060 1      23.2   3.5  157.0  5.6e-02  199.9  131.6   26.5  9.8e-02   
i5      1      30.6  12.4   30.0  1.9e-01  316.7  117.4    9.3  3.7e-02   
knl     1      23.9  14.4  447.0  1.5e+01   78.4   67.3  143.3  2.2e+00   
        2      73.9  14.9  453.4  8.9e+00   99.4   93.9  128.9  8.2e+00   
        4      24.4  13.8  450.0  2.6e+01  132.1  128.5  112.9  5.8e+00   
p100    1       4.1   3.1  552.8  1.0e+00   61.1   56.4  345.8  1.9e+00   
        2       3.5   3.3  554.2  1.2e-01   60.3   56.1  338.2  3.2e+00   
        4       3.2   3.1  554.8  1.7e+00   72.2   68.0  320.3  7.5e+00   
skl     1       6.6   6.1  206.8  3.0e+00   31.4   24.5  193.6  8.6e+00   
        2       8.1   6.9  210.4  6.1e+00   34.1   32.6  184.5  8.4e+00   
        4       8.5   6.0  225.5  1.3e+01   42.5   40.7  173.0  1.2e+01   
v100    1       3.8   3.1  848.3  6.8e-01   42.6   41.5  591.6  2.9e+00   
        2       3.3   3.1  847.9  3.7e-01   50.0   48.2  575.3  7.0e+00   
        4       3.4   3.4  844.9  1.8e+00   62.5   59.2  525.7  1.4e+01   

               dxdy2         ...   dxdy3          dxdy4                       \
                 lat         ...      bw            lat            bw          
                 avg    min  ...     avg    std     avg    min    avg    std   
arch    nodes                ...                                               
gtx1060 1       58.1   11.7  ...   110.6    0.7   322.6   72.5   80.0   15.6   
i5      1      175.2   28.7  ...    29.0    0.7  1212.1  208.3   25.8    1.0   
knl     1       65.5   42.5  ...   161.0   22.3   281.6  188.3   91.7   34.6   
        2      148.2   93.4  ...   120.2   11.2   295.2  216.3   74.1   30.1   
        4      201.0  140.3  ...    95.0   20.4   268.3  222.8   79.2   10.7   
p100    1       26.7    7.4  ...   239.3    1.5   148.6   30.1  200.7    1.7   
        2      477.0  465.0  ...   139.2   40.5   543.7  484.8  146.0   27.9   
        4      687.1  682.0  ...    84.1   38.6   728.4  699.1  102.0   35.0   
skl     1       62.9   41.1  ...   151.7   12.6   374.2   85.6  116.4    3.9   
        2       57.8   48.0  ...   133.8   27.4   167.6   78.3  104.1   17.5   
        4       48.3   45.4  ...   130.1   23.7   102.9   64.6   93.4   15.3   
v100    1       12.0    5.1  ...   723.0    9.7    47.1   12.6  656.3   11.6   
        2      455.0  452.5  ...   261.7  120.3   472.6  452.2  332.8  117.0   
        4      694.8  691.2  ...   119.7   70.4   711.5  695.6  176.2   90.2   

                dxdy5                      
                  lat            bw        
                  avg    min    avg   std  
arch    nodes                              
gtx1060 1       571.4  125.0   61.1  18.8  
i5      1      1956.6  340.4   20.8   1.4  
knl     1       483.5  302.7   92.1   5.8  
        2       460.2  339.6   87.3   4.1  
        4       370.0  289.3   74.9  10.8  
p100    1       273.0   62.7  162.6  18.1  
        2       606.3  491.4  140.2  17.6  
        4       763.3  694.4  108.6  27.5  
skl     1       423.0  132.0   96.7   9.4  
        2       254.1  101.1  100.8  11.0  
        4       149.4   73.8   94.3  17.0  
v100    1        85.5   23.6  522.9  60.7  
        2       492.7  454.8  358.2  94.6  
        4       721.0  688.0  216.5  95.8  

[14 rows x 24 columns]

In [13]:
#arr=arr.reset_index()

In [21]:
#define conversion function 
def toString(x): 
    np.ceil(x)
    #string = '%.1f'% x
    string = '%d' %np.ceil(x)
    return string

In [22]:
addto = []
for n in ['axpby','dot','dxdy2','dxdy3','dxdy4','dxdy5']:
    arr.loc[:,(n,'bw','string')]= arr[n]['bw']['avg'].apply(toString) +" ± "+arr[n]['bw']['std'].apply(toString)
    addto.append((n,'lat','min'))
    addto.append((n,'bw','string'))

#make a table for display
nicetable=arr[addto]
drop = nicetable.columns.droplevel(2)
nicetable.columns=drop
#nicetable.reset_index(inplace=True)
#nicetable.set_index('arch')
newindex=[('i5',1)]
for n in ['skl','knl']:
    for m in [1,2,4]:
        newindex.append((n,m))
newindex.append(('gtx1060',1))
for n in ['p100','v100']:
    for m in [1,2,4]:
        newindex.append((n,m))
    
nicetable=nicetable.reindex(newindex)
nicetable

axpby              dot            dxdy2            dxdy3  \
                lat        bw    lat        bw    lat        bw    lat   
arch    nodes                                                            
i5      1      12.4    30 ± 1  117.4    10 ± 1   28.7    28 ± 3   69.8   
skl     1       6.1   207 ± 3   24.5   194 ± 9   41.1  153 ± 40   50.6   
        2       6.9   211 ± 7   32.6   185 ± 9   48.0  142 ± 33   60.5   
        4       6.0  226 ± 13   40.7  173 ± 13   45.4  114 ± 17   51.2   
knl     1      14.4  447 ± 15   67.3   144 ± 3   42.5  237 ± 20   84.3   
        2      14.9   454 ± 9   93.9   129 ± 9   93.4  152 ± 28  139.2   
        4      13.8  451 ± 27  128.5   113 ± 6  140.3   95 ± 22  171.1   
gtx1060 1       3.5   158 ± 1  131.6    27 ± 1   11.7   131 ± 1   34.5   
p100    1       3.1   553 ± 2   56.4   346 ± 2    7.4   288 ± 3   15.7   
        2       3.3   555 ± 1   56.1   339 ± 4  465.0  108 ± 48  473.1   
        4       3.1   555 ± 2   68.0   321 ± 8  682.0   53 ± 31  678.0   
v100    1       3.1   849 ± 1   41.5   592 ± 3    5.1  802 ± 17    7.8   
        2       3.1   848 ± 1   48.2   576 ± 8  452.5  160 ± 91  455.0   
        4       3.4   845 ± 2   59.2  526 ± 14  691.2   62 ± 40  691.4   

                          dxdy4             dxdy5            
                      bw    lat         bw    lat        bw  
arch    nodes                                                
i5      1         29 ± 1  208.3     26 ± 1  340.4    21 ± 2  
skl     1       152 ± 13   85.6    117 ± 4  132.0   97 ± 10  
        2       134 ± 28   78.3   105 ± 18  101.1  101 ± 11  
        4       131 ± 24   64.6    94 ± 16   73.8   95 ± 17  
knl     1       162 ± 23  188.3    92 ± 35  302.7    93 ± 6  
        2       121 ± 12  216.3    75 ± 31  339.6    88 ± 5  
        4        95 ± 21  222.8    80 ± 11  289.3   75 ± 11  
gtx1060 1        111 ± 1   72.5    80 ± 16  125.0   62 ± 19  
p100    1        240 ± 2   30.1    201 ± 2   62.7  163 ± 19  
        2       140 ± 41  484.8   146 ± 28  491.4  141 ± 18  
        4        85 ± 39  699.1   102 ± 35  694.4  109 ± 28  
v100    1       723 ± 10   12.6   657 ± 12   23.6  523 ± 61  
        2      262 ± 121  452.2  333 ± 118  454.8  359 ± 95  
        4       120 ± 71  695.6   177 ± 91  688.0  217 ± 96

#### Assumptions
- there are three basic functions: trivially parallel(axpby), nearest neighbor (dxdy), global reduction (dot)
- each can be represented by the single node bandwidth, the single node latency and the multinode latency

#### But
- does not capture cache effect e.g. in SKl

In [29]:
index = ['i5','skl','knl','gtx1060','v100','p100']  
lines = []
for arch in  index: 
    line = []
    line.append(arch)
    #first the bandwidths
    line.append( arr.loc[(arch,1),('axpby','bw','avg')] )
    for n in ['dot','dxdy2','dxdy3','dxdy4','dxdy5']:
        line.append( arr.loc[(arch,1),(n,'bw','avg')] /line[1])
    for n in ['axpby','dot','dxdy2'] :
        line.append( arr.loc[(arch,1),(n,'lat','min')] )
        if arch == 'i5' or arch == 'gtx1060':
            line.append(None)
        else:
            line.append( arr.loc[(arch,4),(n,'lat','min')] )
    lines.append(line)
    
tuples=['arch']     

for n in ['axpby','dot','dxdy2','dxdy3','dxdy4','dxdy5']:
    tuples.append(n+'_bw')
for n in ['axpby','dot','dxdy']:
    tuples.append(n+'_lat_shared')
    tuples.append(n+'_lat_dist')
cols=tuples
toDisk = pd.DataFrame(lines, columns=cols)
toDisk.to_csv('performance.csv',sep=' ',index=False)

In [30]:
test = pd.read_csv('performance.csv',delimiter=' ')
test

,arch,axpby_bw,dot_bw,dxdy2_bw,dxdy3_bw,dxdy4_bw,dxdy5_bw,axpby_lat_shared,axpby_lat_dist,dot_lat_shared,dot_lat_dist,dxdy_lat_shared,dxdy_lat_dist
0,i5,30.0,0.3,0.9,1.0,0.9,0.7,12.4,NaN,117.4,NaN,28.7,NaN
1,skl,206.8,0.9,0.7,0.7,0.6,0.5,6.1,6.0,24.5,40.7,41.1,45.4
2,knl,447.0,0.3,0.5,0.4,0.2,0.2,14.4,13.8,67.3,128.5,42.5,140.3
3,gtx1060,157.0,0.2,0.8,0.7,0.5,0.4,3.5,NaN,131.6,NaN,11.7,NaN
4,v100,848.3,0.7,0.9,0.9,0.8,0.6,3.1,3.4,41.5,59.2,5.1,691.2
5,p100,552.8,0.6,0.5,0.4,0.4,0.3,3.1,3.1,56.4,68.0,7.4,682.0
